<h4> VALIDACION CRUZDA CON RANDOM SEARCH Y GRID SEARCH</h4>
<p>En este archivo se muestra como realizar una validacion cruzada con Random Search y Grid Search.</p>

In [1]:
import pandas as pd # importamos pandas para manipular los datos
from sklearn.ensemble import RandomForestRegressor # importamos RandomForestRegressor para crear el modelo
from sklearn.model_selection import RandomizedSearchCV # importamos RandomizedSearchCV para la búsqueda de hiperparámetros
import numpy as np # importamos numpy para operaciones numéricas
import matplotlib.pyplot as plt # importamos matplotlib para graficar los resultados

In [3]:
# Cargar datos
df_train = pd.read_csv('../data/train_spotify_tracks.csv')
df_test = pd.read_csv('../data/test_spotify_tracks.csv')
y_test = pd.read_csv('../data/y_test.csv')

X_train = df_train.drop('popularity', axis=1)
y_train = df_train['popularity']

In [4]:
import time
# Modelo base
modelo = RandomForestRegressor(random_state=42)

# Distribución de hiperparámetros
param_dist = {
    'n_estimators': [100, 200],       # ↓ Menos árboles
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_depth': [10, 20, 30]         # ↓ Más profundidad
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=modelo,
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=1,
    verbose=1
)

# Entrenamiento y cronómetro
print("⏳ Entrenando modelo con RandomizedSearchCV...")
start = time.time()
random_search.fit(X_train, y_train)
end = time.time()
print(f"\n✅ Entrenamiento finalizado en {end - start:.2f} segundos.")

# Mostrar la mejor combinación
print("\n🎯 Mejor combinación de hiperparámetros encontrados:")
print(random_search.best_params_)

⏳ Entrenando modelo con RandomizedSearchCV...
Fitting 5 folds for each of 10 candidates, totalling 50 fits

✅ Entrenamiento finalizado en 2480.62 segundos.

🎯 Mejor combinación de hiperparámetros encontrados:
{'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': 20}


In [5]:
import pandas as pd

# Crear DataFrame con los resultados de la búsqueda
resultados = pd.DataFrame(random_search.cv_results_)

# Seleccionar columnas relevantes
cols = [
    'params',
    'mean_test_score',
    'std_test_score',
    'rank_test_score'
]
df_resultados = resultados[cols].copy()

# Cambiar el nombre de las columnas para mayor claridad
df_resultados.rename(columns={
    'params': 'Hiperparámetros',
    'mean_test_score': 'MSE (negativo)',
    'std_test_score': 'STD MSE',
    'rank_test_score': 'Ranking'
}, inplace=True)

# Ordenar por el mejor MSE (más cercano a 0, ya que es negativo)
df_resultados = df_resultados.sort_values('MSE (negativo)', ascending=False).reset_index(drop=True)

# Mostrar el DataFrame
df_resultados

,Hiperparámetros,MSE (negativo),STD MSE,Ranking
0,"{'n_estimators': 200, 'min_samples_split': 2, ...",-246.292925,3.026692,1
1,"{'n_estimators': 200, 'min_samples_split': 5, ...",-247.232234,2.881295,2
2,"{'n_estimators': 200, 'min_samples_split': 2, ...",-247.747522,2.840440,3
3,"{'n_estimators': 200, 'min_samples_split': 2, ...",-248.143055,2.824805,4
4,"{'n_estimators': 100, 'min_samples_split': 2, ...",-248.790545,2.778526,5
5,"{'n_estimators': 100, 'min_samples_split': 5, ...",-250.572424,2.613065,6
6,"{'n_estimators': 100, 'min_samples_split': 2, ...",-252.544418,2.507026,7
7,"{'n_estimators': 200, 'min_samples_split': 2, ...",-265.092180,3.706107,8
8,"{'n_estimators': 200, 'min_samples_split': 2, ...",-265.267312,3.737299,9
9,"{'n_estimators': 100, 'min_samples_split': 2, ...",-265.548094,3.631106,10


In [7]:
# descargar el csv con los resultados
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('../output/random_search_results.csv', index=False)
